In [1]:
%load_ext autotime

In [2]:
import boto3
import datetime
import logging
import json
import pickle
import pytest
import sys
import time

from etltools import s3

reload(logging)  # get around notebook problem

<module 'logging' from '/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/logging/__init__.pyc'>

time: 1.1 s


In [3]:
logging.basicConfig(
    level=logging.INFO, 
    format='[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
#     handlers=[
#         logging.FileHandler(filename='mylog.log', mode='w'),
#         logging.StreamHandler(sys.stdout),
#     ]
)

time: 1.35 ms


In [4]:
# Test whether logging works:
logger = logging.getLogger()
logger.info('TEST INFO')

[2018-01-11 15:05:45,611] {<ipython-input-4-efd2929c73d3>:3} INFO - TEST INFO


time: 2.02 ms


In [5]:
aws_info = {
    'profile_name': 'sandbox',
}

uids = s3.read_json('s3://picwell.sandbox.medicare/samples/philadelphia-2015-1k-sample')
pids = ['2820028008119', '2820088001036']

time: 303 ms


# Test ConfigInfo

In [6]:
from lambda_client.config_info import ConfigInfo

configs = ConfigInfo('lambda_client/lambda.cfg')

print configs.claims_bucket
print configs.claims_path
print
print configs.benefits_bucket
print configs.benefits_path
print
print configs.claims_table

picwell.sandbox.analytics
junghoon/lambda_calculator

picwell.sandbox.analytics
junghoon/lambda_calculator_benefits

ma_claims
time: 17.2 ms


In [7]:
all_states = configs.all_states

print '{} states'.format(len(all_states))
print all_states

51 states
['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.16 ms


# Test ClaimsClient

In [8]:
from lambda_client import ClaimsClient

time: 770 µs


In [9]:
# Test S3:
client = ClaimsClient(aws_info, 
                      s3_bucket=configs.claims_bucket,
                      s3_path=configs.claims_path)

people = client.get(uids[:1])
print 'claims of {} people retrieved'.format(len(people))

[2018-01-11 15:05:45,961] {shared_utils.py:64} INFO - Thread initialization took 0.003543 seconds.
[2018-01-11 15:05:46,249] {shared_utils.py:64} INFO - Joining all threads took 0.283701 seconds.
[2018-01-11 15:05:46,250] {shared_utils.py:64} INFO - Combining all results took 2.2e-05 seconds.


claims of 1 people retrieved
time: 299 ms


In [10]:
person = people[0]
print person.keys()
{
    'uid': person['uid'],
    'medical_claims': person['medical_claims'][:5]
}

[u'medical_claims', u'uid']


{'medical_claims': [{u'admitted': u'2014-04-03',
   u'benefit_category': 16,
   u'cost': u'148.0',
   u'discharged': u'2014-04-03',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-02',
   u'benefit_category': 16,
   u'cost': u'74.55',
   u'discharged': u'2014-05-02',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-05',
   u'benefit_category': 16,
   u'cost': u'104.39',
   u'discharged': u'2014-05-05',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-13',
   u'benefit_category': 16,
   u'cost': u'210.12',
   u'discharged': u'2014-05-13',
   u'length_of_stay': 1},
  {u'admitted': u'2014-05-19',
   u'benefit_category': 11,
   u'cost': u'442.2',
   u'discharged': u'2014-05-19',
   u'length_of_stay': 1}],
 'uid': u'1302895801'}

time: 5.87 ms


In [11]:
# Test DynamoDB:
client = ClaimsClient(aws_info,
                      table_name=configs.claims_table)

people = client.get(uids[:1])
print 'claims of {} people retrieved'.format(len(people))

[2018-01-11 15:05:46,265] {shared_utils.py:64} INFO - Thread initialization took 0.000574 seconds.
[2018-01-11 15:05:46,516] {shared_utils.py:64} INFO - Joining all threads took 0.247355 seconds.
[2018-01-11 15:05:46,517] {shared_utils.py:64} INFO - Combining all results took 2.4e-05 seconds.


claims of 1 people retrieved
time: 255 ms


In [12]:
# Test configuration file and retrieving multiple people:
client = ClaimsClient(aws_info)

people = client.get(uids[:5])
print 'claims of {} people retrieved'.format(len(people))

[2018-01-11 15:05:46,540] {shared_utils.py:64} INFO - Thread initialization took 0.016502 seconds.
[2018-01-11 15:05:47,370] {shared_utils.py:64} INFO - Joining all threads took 0.825435 seconds.
[2018-01-11 15:05:47,371] {shared_utils.py:64} INFO - Combining all results took 5.7e-05 seconds.


claims of 5 people retrieved
time: 850 ms


In [13]:
# Let's try something larger:
people = client.get(uids)
print 'claims of {} people retrieved'.format(len(people))

[2018-01-11 15:08:32,637] {shared_utils.py:64} INFO - Thread initialization took 165.26082 seconds.
[2018-01-11 15:08:45,694] {shared_utils.py:64} INFO - Joining all threads took 13.023948 seconds.
[2018-01-11 15:08:45,699] {shared_utils.py:64} INFO - Combining all results took 0.004113 seconds.


claims of 1000 people retrieved
time: 2min 58s


In [14]:
# The object should not be pickled.
with pytest.raises(Exception, match='ClaimsClient object cannot be pickled.'):
    pickle.dumps(client)

time: 1.47 ms


# Test BenefitsClient

In [15]:
from lambda_client import BenefitsClient

time: 1.14 ms


In [16]:
client = BenefitsClient(aws_info)

print client.all_states

['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 1.85 ms


In [17]:
plans = client._get_one_state('01')
print '{} plans read for state 01'.format(len(plans))

plans = client._get_one_state('04')
print '{} plans read for state 04'.format(len(plans))

47 plans read for state 01
75 plans read for state 04
time: 1.11 s


In [18]:
plans = client.get_by_state(['01', '04'])
print '{} plans read'.format(len(plans))

[2018-01-11 15:08:46,842] {shared_utils.py:64} INFO - Thread initialization took 0.001461 seconds.
[2018-01-11 15:08:47,508] {shared_utils.py:64} INFO - Joining all threads took 0.662049 seconds.
[2018-01-11 15:08:47,509] {shared_utils.py:64} INFO - Combining all results took 2.6e-05 seconds.


122 plans read
time: 671 ms


In [19]:
plans = client.get_all()
print '{} plans read'.format(len(plans))

[2018-01-11 15:08:47,729] {shared_utils.py:64} INFO - Thread initialization took 0.212862 seconds.
[2018-01-11 15:08:59,798] {shared_utils.py:64} INFO - Joining all threads took 12.061444 seconds.
[2018-01-11 15:08:59,799] {shared_utils.py:64} INFO - Combining all results took 0.000267 seconds.


3558 plans read
time: 12.3 s


In [20]:
# Compare the timing against reading the entire file:
from lambda_client.shared_utils import _read_json

session = boto3.Session(**aws_info)
resource = session.resource('s3')

time: 81.6 ms


In [21]:
all_plans = _read_json('picwell.sandbox.medicare', 'ma_benefits/cms_2018_pbps_20171005.json', resource)

print '{} plans read'.format(len(plans))

3558 plans read
time: 3.72 s


In [22]:
# Ensure that the same plans are read:
sort_key = lambda plan: plan['picwell_id']
assert sorted(all_plans, key=sort_key) == sorted(plans, key=sort_key)

time: 179 ms


In [23]:
# The object should not be pickled.
with pytest.raises(Exception, match='BenefitsClient object cannot be pickled.'):
    pickle.dumps(client)

time: 1.72 ms


# Test Cost Breakdown

In [24]:
from lambda_client import CalculatorClient

client = CalculatorClient(aws_info)

time: 902 µs


In [25]:
responses = client._get_one_breakdown(uids[0], pids, '01', True)

print '{} responses returned'.format(len(responses))
responses[0]

Cost breakdown complete (elapsed: 6.02472 seconds).
2 responses returned


{u'allowed': 26376.640000000003,
 u'covered_breakdown': {u'categories': {u'0': 0.0,
   u'11': 186.33999999999997,
   u'13': 13.23,
   u'15': 30.79,
   u'16': 0.0,
   u'19': 80.0,
   u'25': 1750.0,
   u'30': 719.1700000000001,
   u'31': 300.0,
   u'44': 0.0,
   u'49': 118.48800000000001,
   u'7': 479.558},
  u'composite': 3677.5759999999996,
  u'in_network': 3677.5759999999996,
  u'out_network': 0.0},
 u'deductible_breakdown': {u'categories': {u'0': 0.0,
   u'11': 0.0,
   u'13': 0.0,
   u'15': 0.0,
   u'16': 0.0,
   u'19': 0.0,
   u'25': 0.0,
   u'30': 0.0,
   u'31': 0.0,
   u'44': 0.0,
   u'49': 0.0,
   u'7': 0.0},
  u'composite': 0.0,
  u'in_network': 0.0,
  u'out_network': 0.0},
 u'oop': 3677.5759999999996,
 u'picwell_id': u'2820028008119',
 u'uid': u'1302895801',
 u'uncovered': 0.0,
 u'uncovered_breakdown': {u'categories': {u'0': 0.0,
   u'11': 0.0,
   u'13': 0.0,
   u'15': 0.0,
   u'16': 0.0,
   u'19': 0.0,
   u'25': 0.0,
   u'30': 0.0,
   u'31': 0.0,
   u'44': 0.0,
   u'49': 0.0,


time: 6.56 s


In [26]:
responses = client.get_breakdown(uids[:1], pids, verbose=True)

print '{} responses returned'.format(len(responses))

[2018-01-11 15:09:10,382] {shared_utils.py:64} INFO - Thread initialization took 0.000595 seconds.
[2018-01-11 15:09:16,713] {shared_utils.py:64} INFO - Joining all threads took 6.329885 seconds.
[2018-01-11 15:09:16,714] {shared_utils.py:64} INFO - Combining all results took 2.4e-05 seconds.


Cost breakdown complete (elapsed: 6.193842 seconds).
2 responses returned
time: 6.33 s


In [27]:
# It seems like
#
# (a) the time it takes to issue threads increases linearly; and
# (b) the time it takes for the last thread to finish is about 5 seconds all the time. 
for num_people in range(2, 15):
    start = datetime.datetime.now()
    responses = client.get_breakdown(uids[:num_people], pids)
    elapsed = (datetime.datetime.now() - start).total_seconds()

    print '{} responses returned ({} seconds)'.format(len(responses), elapsed)

[2018-01-11 15:09:16,723] {shared_utils.py:64} INFO - Thread initialization took 0.001867 seconds.
[2018-01-11 15:09:23,753] {shared_utils.py:64} INFO - Joining all threads took 7.025948 seconds.
[2018-01-11 15:09:23,754] {shared_utils.py:64} INFO - Combining all results took 3e-05 seconds.
[2018-01-11 15:09:23,758] {shared_utils.py:64} INFO - Thread initialization took 0.003336 seconds.
[2018-01-11 15:09:30,216] {shared_utils.py:64} INFO - Joining all threads took 6.454736 seconds.
[2018-01-11 15:09:30,217] {shared_utils.py:64} INFO - Combining all results took 3e-05 seconds.


4 responses returned (7.033753 seconds)
6 responses returned (6.462928 seconds)

[2018-01-11 15:09:30,224] {shared_utils.py:64} INFO - Thread initialization took 0.005772 seconds.
[2018-01-11 15:09:37,196] {shared_utils.py:64} INFO - Joining all threads took 6.968006 seconds.
[2018-01-11 15:09:37,196] {shared_utils.py:64} INFO - Combining all results took 4.1e-05 seconds.



8 responses returned (6.978946 seconds)

[2018-01-11 15:09:37,205] {shared_utils.py:64} INFO - Thread initialization took 0.008165 seconds.
[2018-01-11 15:09:44,362] {shared_utils.py:64} INFO - Joining all threads took 7.150705 seconds.
[2018-01-11 15:09:44,363] {shared_utils.py:64} INFO - Combining all results took 6.7e-05 seconds.


...


24 responses returned (7.697395 seconds)

[2018-01-11 15:10:37,570] {shared_utils.py:64} INFO - Thread initialization took 0.133602 seconds.
[2018-01-11 15:10:45,168] {shared_utils.py:64} INFO - Joining all threads took 7.584413 seconds.
[2018-01-11 15:10:45,168] {shared_utils.py:64} INFO - Combining all results took 6.9e-05 seconds.



26 responses returned (7.732669 seconds)

[2018-01-11 15:10:45,266] {shared_utils.py:64} INFO - Thread initialization took 0.096293 seconds.
[2018-01-11 15:10:52,238] {shared_utils.py:64} INFO - Joining all threads took 6.869489 seconds.
[2018-01-11 15:10:52,239] {shared_utils.py:64} INFO - Combining all results took 7e-05 seconds.



28 responses returned (7.069777 seconds)
time: 1min 35s


In [28]:
# Let's try something larger:
responses = client.get_breakdown(uids, pids)

print '{} responses returned'.format(len(responses))

[2018-01-11 15:12:40,158] {shared_utils.py:64} INFO - Thread initialization took 107.913674 seconds.
[2018-01-11 15:12:51,014] {shared_utils.py:64} INFO - Joining all threads took 9.761013 seconds.
[2018-01-11 15:12:51,019] {shared_utils.py:64} INFO - Combining all results took 0.004265 seconds.


2000 responses returned
time: 1min 58s


In [29]:
# Run calculcations locally for comparison:
from lambda_package.calc.calculator import calculate_oop

claims_client = ClaimsClient(aws_info)
people = claims_client.get(uids)

benefits_client = BenefitsClient(aws_info)
plans = benefits_client.get_by_pid(pids)

costs = []
for person in people:
    claims = person['medical_claims']
    
    for plan in plans:
        cost = calculate_oop(claims, plan)
        cost.update({
            'uid': person['uid'],
            'picwell_id': str(plan['picwell_id']),
        })
        
        costs.append(cost)
        
print '{} costs calculated'.format(len(costs))

[2018-01-11 15:16:08,403] {shared_utils.py:64} INFO - Thread initialization took 197.320774 seconds.
[2018-01-11 15:16:23,670] {shared_utils.py:64} INFO - Joining all threads took 15.158315 seconds.
[2018-01-11 15:16:23,677] {shared_utils.py:64} INFO - Combining all results took 0.005958 seconds.
[2018-01-11 15:16:23,714] {shared_utils.py:64} INFO - Thread initialization took 0.0032 seconds.
[2018-01-11 15:16:25,531] {shared_utils.py:64} INFO - Joining all threads took 1.811888 seconds.
[2018-01-11 15:16:25,532] {shared_utils.py:64} INFO - Combining all results took 3.2e-05 seconds.


2000 costs calculated
time: 3min 36s


# Test Batch Calculation

In [30]:
# uids = s3.read_json('s3n://picwell.sandbox.medicare/samples/philadelphia-2015')

# print '{} uids read'.format(len(uids))

time: 967 µs


In [31]:
# uids[:10]

time: 791 µs


In [32]:
# requests_per_second = 100

# for uid in uids:
# #     client.calculate_async(uid, months=['01'])
#     client.calculate_async(uid)
#     time.sleep(1.0/requests_per_second)  

time: 1.04 ms
